<a href="https://colab.research.google.com/github/Sachinikuruppu/Web-Content-Clustering/blob/main/WebClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk

# Sample emails dataset
emails = [
    "Subject:Project Update.	Hi Jane, Just wanted to give you a quick update on the project. We're making good progress and should be able to meet the deadline. Best, John",
    "Subject:Re: Project Update.	Hi John, Thanks for the update. That's great to hear. Let me know if you need any help. Regards, Jane",
    "Subject:Team Meeting Reminder.	Hi Team, Just a reminder that we have a meeting scheduled for tomorrow at 10 AM. Please be on time. Thanks, Mark",
    "Subject:Re: Team Meeting Reminder.	Hi Mark, Noted. I'll be there. Best, Susan",
    "Subject:Budget Report.	Hi Bob, Attached is the budget report for this quarter. Let me know if you have any questions. Best, Alice",
    "Subject:Re: Budget Report.	Hi Alice, Thanks for the report. I'll review it and get back to you if I have any questions. Regards, Bob",
    "Subject:New Marketing Strategy.	Hi David, I've drafted a new marketing strategy. Please review and provide your feedback. Thanks, Carol",
    "Subject:Re: New Marketing Strategy.	Hi Carol, The new strategy looks good to me. Let's discuss further in our next meeting. Regards, David",
    "Subject:Website Redesign.	Hi Frank, We're planning a redesign of the company website. Can you send me your ideas by end of this week? Thanks, Eve",
    "Subject:Re: Website Redesign.	Hi Eve, Sure, I'll send you my ideas by Friday. Best, Frank",
    "Subject:Client Meeting Recap.	Hi Hannah, Here's a recap of our meeting with the client yesterday. Let me know if I missed anything. Thanks, George",
    "Subject:Re: Client Meeting Recap.	Hi George, Thanks for the recap. Everything looks good. Best, Hannah",
    "Subject:Monthly Sales Report.	Hi Julie, The monthly sales report is ready. I've attached it to this email. Regards, Ian",
    "Subject:Re: Monthly Sales Report.	Hi Ian, Thanks for the report. I'll review it today. Best, Julie",
    "Subject:Annual Leave Request.	Hi Larry, I'd like to request annual leave from August 1st to August 10th. Please approve. Thanks, Karen",
    "Subject:Re: Annual Leave Request.	Hi Karen, Your leave request is approved. Have a great vacation. Best, Larry",
    "Subject:Invoice for Services.	Hi Nancy, Please find attached the invoice for the services rendered last month. Best, Mike",
    "Subject:Re: Invoice for Services.	Hi Mike, I've received the invoice. I'll process the payment by the end of the week. Thanks, Nancy",
    "Subject:Product Launch Plan.	Hi Paula, Here's the plan for the upcoming product launch. Please review and provide your feedback. Regards, Oliver",
    "Subject:Re: Product Launch Plan.	Hi Oliver, The plan looks good. Let's move forward with it. Best, Paula",
    "Subject:Training Session Confirmation.	Hi Rachel, This is to confirm your training session scheduled for next Monday at 2 PM. Regards, Quinn",
    "Subject:Re: Training Session Confirmation.	Hi Quinn, Thanks for the confirmation. I'll be there. Best, Rachel",
    "Subject:Marketing Budget Approval.	Hi Tina, Please review and approve the attached marketing budget for next quarter. Thanks, Sam",
    "Subject:Re: Marketing Budget Approval.	Hi Sam, The budget looks good to me. Approved. Regards, Tina",
    "Subject:Partnership Proposal.	Hi Victor, We're interested in a potential partnership with your company. Please review the attached proposal. Best, Ursula",
    "Subject:Re: Partnership Proposal.	Hi Ursula, Thank you for the proposal. We're interested as well. Let's set up a meeting to discuss further. Regards, Victor",
    "Subject:Project Timeline.	Hi Xena, Attached is the timeline for the new project. Please review and let me know if you have any concerns. Best, Will",
    "Subject:Re: Project Timeline.	Hi Will, The timeline looks good. No concerns from my side. Regards, Xena",
    "Subject:Team Outing Details.	Hi Zach, Here are the details for our team outing next month. Please confirm your attendance. Thanks, Yara",
    "Subject:Re: Team Outing Details.	Hi Yara, I'm looking forward to it. I'll be there. Best, Zach",
    "Subject:Technical Issue Reported.	Hi Betty, We've encountered a technical issue with the server. Please look into it. Regards, Andy",
    "Subject:Re: Technical Issue Reported.	Hi Andy, I'm on it. I'll update you once it's resolved. Thanks, Betty",
    "Subject:Client Feedback.	Hi Donna, We've received feedback from the client regarding our recent project. Please review and take necessary action. Regards, Charles",
    "Subject:Re: Client Feedback.	Hi Charles, I'll review the feedback and follow up with the client. Thanks, Donna",
    "Subject:Contract Renewal.	Hi Fiona, Our contract is up for renewal next month. Please review the terms and let me know if there are any changes. Regards, Edward",
    "Subject:Re: Contract Renewal.	Hi Edward, I'll review the contract and get back to you by the end of this week. Best, Fiona",
    "Subject:Sales Target Achievement.	Hi Hannah, Great news! We've achieved our sales target for this quarter. Let's keep up the good work. Regards, George",
    "Subject:Re: Sales Target Achievement.	Hi George, That's fantastic news! Congratulations to the team. Best, Hannah",
    "Subject:Employee Performance Review.	Hi Julie, Your performance review is scheduled for next Wednesday at 3 PM. Regards, Ian",
    "Subject:Re: Employee Performance Review.	Hi Ian, Thank you for the information. I'll be there. Regards, Julie",
    "Subject:Marketing Campaign Results.	Hi Leo, The results of our recent marketing campaign are in. Please review the attached report. Regards, Kate",
    "Subject:Re: Marketing Campaign Results.	Hi Kate, Thanks for the report. I'll review it today. Best, Leo",
    "Subject:Software Update.	Hi Nancy, We've released a new update for the software. Please make sure to install it as soon as possible. Regards, Matt",
    "Subject:Re: Software Update.	Hi Matt, I'll install the update right away. Thanks for the heads-up. Best, Nancy",
    "Subject:Conference Call.	Hi Patricia, We have a conference call with the client tomorrow at 3 PM. Please make sure to join. Regards, Oscar",
    "Subject:Re: Conference Call.	Hi Oscar, I'll be there. Thanks for the reminder. Best, Patricia",
    "Subject:Budget Approval.	Hi Robin, Can you please review and approve the budget for next quarter? Thanks, Quincy",
    "Subject:Re: Budget Approval.	Hi Quincy, I've reviewed the budget and it looks good. Approved. Regards, Robin",
    "Subject:Meeting Agenda.	Hi Tom, Attached is the agenda for our meeting tomorrow. Please review and let me know if you have any additions. Regards, Sophie",
    "Subject:Re: Meeting Agenda.	Hi Sophie, The agenda looks good to me. No additions from my side. Best, Tom",
    "Subject:Quarterly Review.	Hi Victor, Our quarterly review meeting is scheduled for next Monday. Please prepare your report. Regards, Ursula",
    "Subject:Re: Quarterly Review.	Hi Ursula, I'll have my report ready by then. Thanks for the reminder. Best, Victor",
    "Subject:Project Kickoff.	Hi Xander, We're kicking off the new project next week. Please prepare the necessary documents. Regards, Wendy",
    "Subject:Re: Project Kickoff.	Hi Wendy, I'll have everything ready for the kickoff. Thanks, Xander",
    "Subject:Sales Presentation.	Hi Zach, Please find attached the presentation for the sales meeting tomorrow. Regards, Yara",
    "Subject:Re: Sales Presentation.	Hi Yara, Thanks for the presentation. I'll review it today. Best, Zach",
    "Subject:Server Maintenance.	Hi Betty, We need to schedule a maintenance window for the server next week. Please suggest a suitable time. Regards, Andy",
    "Subject:Re: Server Maintenance.	Hi Andy, How about next Wednesday at 8 PM? Best, Betty",
    "Subject:Client Meeting.	Hi Donna, We have a meeting with the client on Thursday at 10 AM. Please be prepared with the project updates. Regards, Charles",
    "Subject:Re: Client Meeting.	Hi Charles, I'll have the updates ready. Thanks for the reminder. Best, Donna",
    "Subject:Training Session.	Hi Fiona, Just a reminder about the training session on Friday at 2 PM. Regards, Edward",
    "Subject:Re: Training Session.	Hi Edward, Thanks for the reminder. I'll be there. Best, Fiona",
    "Subject:New Hire Onboarding.	Hi Hannah, The new hire will start on Monday. Please prepare the onboarding documents. Regards, George",
    "Subject:Re: New Hire Onboarding.	Hi George, I'll have everything ready for the new hire. Thanks, Hannah",
    "Subject:Quarterly Sales Meeting.	Hi Julie, Our quarterly sales meeting is scheduled for next Tuesday. Please prepare your sales report. Regards, Ian",
    "Subject:Re: Quarterly Sales Meeting.	Hi Ian, I'll have the report ready by then. Best, Julie",
    "Subject:Market Research.	Hi Leo, I've completed the market research for the new product. Please review the attached report. Regards, Kate",
    "Subject:Re: Market Research.	Hi Kate, Thanks for the report. I'll review it and get back to you. Best, Leo",
    "Subject:Customer Feedback.	Hi Nancy, We've received some customer feedback about our product. Please review the attached document. Regards, Matt",
    "Subject:Re: Customer Feedback.	Hi Matt, Thanks for the feedback. I'll review it and see what improvements we can make. Best, Nancy",
    "Subject:Project Milestone.	Hi Patricia, We've reached an important milestone in the project. Please update the client. Regards, Oscar",
    "Subject:Re: Project Milestone.	Hi Oscar, I'll update the client today. Thanks, Patricia",
    "Subject:Product Feedback.	Hi Robin, We've received feedback from the beta testers. Please review and let me know your thoughts. Regards, Quincy",
    "Subject:Re: Product Feedback.	Hi Quincy, I'll review the feedback and get back to you. Best, Robin",
    "Subject:Press Release.	Hi Tom, Attached is the draft press release for the new product launch. Please review and provide your feedback. Regards, Sophie",
    "Subject:Re: Press Release.	Hi Sophie, The draft looks good. I have no additional comments. Best, Tom",
    "Subject:Team Meeting Follow-up.	Hi Victor, Just a follow-up on our team meeting. Please send your action items by end of day. Regards, Ursula",
    "Subject:Re: Team Meeting Follow-up.	Hi Ursula, I'll send my action items by today. Thanks, Victor",
    "Subject:Event Planning.	Hi Xander, We're planning an event for next month. Please start the preparations and keep me updated. Regards, Wendy",
    "Subject:Re: Event Planning.	Hi Wendy, I'll start the preparations and update you regularly. Best, Xander",
    "Subject:Product Demo.	Hi Zach, Please prepare a demo of the new product for our meeting with the client next week. Regards, Yara",
    "Subject:Re: Product Demo.	Hi Yara, I'll have the demo ready for the meeting. Thanks, Zach",
    "Subject:System Upgrade.	Hi Betty, We're planning a system upgrade next month. Please review the plan and let me know if you have any concerns. Regards, Andy",
    "Subject:Re: System Upgrade.	Hi Andy, The plan looks good to me. No concerns from my side. Best, Betty",
    "Subject:Marketing Collateral.	Hi Donna, Attached is the marketing collateral for the upcoming campaign. Please review and provide your feedback. Regards, Charles",
    "Subject:Re: Marketing Collateral.	Hi Charles, The collateral looks great. No changes needed. Best, Donna",
    "Subject:Meeting Reschedule.	Hi Fiona, We need to reschedule our meeting from Wednesday to Thursday. Please confirm your availability. Regards, Edward",
    "Subject:Re: Meeting Reschedule.	Hi Edward, Thursday works for me. Thanks for letting me know. Best, Fiona",
    "Subject:Client Follow-up.	Hi Hannah, Please follow up with the client regarding the pending invoice. Thanks, George",
    "Subject:Re: Client Follow-up.	Hi George, I'll follow up with the client today. Best, Hannah",
    "Subject:Performance Improvement Plan.	Hi Julie, Attached is your performance improvement plan. Please review and schedule a meeting to discuss. Regards, Ian",
    "Subject:Re: Performance Improvement Plan.	Hi Ian, I'll review the plan and schedule a meeting for next week. Best, Julie",
    "Subject:Client Proposal.	Hi Leo, Attached is the proposal for our new client. Please review and provide your feedback. Regards, Kate",
    "Subject:Re: Client Proposal.	Hi Kate, The proposal looks good. No changes from my side. Best, Leo",
    "Subject:Employee Survey.	Hi Nancy, We've received the results of the employee survey. Please review and prepare a summary. Regards, Matt",
    "Subject:Re: Employee Survey.	Hi Matt, I'll prepare the summary and send it to you by end of day. Best, Nancy",
    "Subject:Project Budget Adjustment.	Hi Patricia, We need to adjust the budget for the current project. Please review the attached document and provide your input. Regards, Oscar",
    "Subject:Re: Project Budget Adjustment.	Hi Oscar, I'll review the document and provide my input by tomorrow. Best, Patricia",
    "Subject:New Software Implementation.	Hi Robin, We're implementing new software next month. Please review the implementation plan and provide your feedback. Regards, Quincy",
    "Subject:Re: New Software Implementation.	Hi Quincy, The implementation plan looks good to me. No feedback from my side. Best, Robin"


]

# Ensure you have downloaded the required nltk data
nltk.download('punkt')
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# TF - Term Frequency
vectorizer_tf = CountVectorizer()
X_tf = vectorizer_tf.fit_transform(emails)
df_tf = pd.DataFrame(X_tf.toarray(), columns=vectorizer_tf.get_feature_names_out())
print("Term Frequency (TF):")
print(df_tf)


Term Frequency (TF):
    10  10th  1st  able  about  achieved  achievement  action  additional  \
0    0     0    0     1      0         0            0       0           0   
1    0     0    0     0      0         0            0       0           0   
2    1     0    0     0      0         0            0       0           0   
3    0     0    0     0      0         0            0       0           0   
4    0     0    0     0      0         0            0       0           0   
..  ..   ...  ...   ...    ...       ...          ...     ...         ...   
95   0     0    0     0      0         0            0       0           0   
96   0     0    0     0      0         0            0       0           0   
97   0     0    0     0      0         0            0       0           0   
98   0     0    0     0      0         0            0       0           0   
99   0     0    0     0      0         0            0       0           0   

    additions  ...  with  work  works  xander  xena  y

In [3]:
# TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(emails)
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())
print("TF-IDF:")
print(df_tfidf)


TF-IDF:
          10  10th  1st      able  about  achieved  achievement  action  \
0   0.000000   0.0  0.0  0.233115    0.0       0.0          0.0     0.0   
1   0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
2   0.247849   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
3   0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
4   0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
..       ...   ...  ...       ...    ...       ...          ...     ...   
95  0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
96  0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
97  0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
98  0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   
99  0.000000   0.0  0.0  0.000000    0.0       0.0          0.0     0.0   

    additional  additions  ...  with  work  works  xander  xena  yara  \
0          0.0    

In [4]:
# Word Count (similar to TF)
vectorizer_wc = CountVectorizer()
X_wc = vectorizer_wc.fit_transform(emails)
df_wc = pd.DataFrame(X_wc.toarray(), columns=vectorizer_wc.get_feature_names_out())
print("Word Count:")
print(df_wc)


Word Count:
    10  10th  1st  able  about  achieved  achievement  action  additional  \
0    0     0    0     1      0         0            0       0           0   
1    0     0    0     0      0         0            0       0           0   
2    1     0    0     0      0         0            0       0           0   
3    0     0    0     0      0         0            0       0           0   
4    0     0    0     0      0         0            0       0           0   
..  ..   ...  ...   ...    ...       ...          ...     ...         ...   
95   0     0    0     0      0         0            0       0           0   
96   0     0    0     0      0         0            0       0           0   
97   0     0    0     0      0         0            0       0           0   
98   0     0    0     0      0         0            0       0           0   
99   0     0    0     0      0         0            0       0           0   

    additions  ...  with  work  works  xander  xena  yara  yest

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

# Vectorize the emails using CountVectorizer
vectorizer_wc = CountVectorizer()
X_wc = vectorizer_wc.fit_transform(emails)

# Convert the sparse matrix to a DataFrame
df_wc = pd.DataFrame(X_wc.toarray(), columns=vectorizer_wc.get_feature_names_out())

# Normalize the DataFrame
df_normalized = pd.DataFrame(normalize(df_wc, axis=0), columns=vectorizer_wc.get_feature_names_out())

print("Normalized Data:")
print(df_normalized)



Normalized Data:
          10  10th  1st  able  about  achieved  achievement  action  \
0   0.000000   0.0  0.0   1.0    0.0       0.0          0.0     0.0   
1   0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
2   0.707107   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
3   0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
4   0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
..       ...   ...  ...   ...    ...       ...          ...     ...   
95  0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
96  0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
97  0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
98  0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   
99  0.000000   0.0  0.0   0.0    0.0       0.0          0.0     0.0   

    additional  additions  ...  with  work  works  xander  xena  yara  \
0          0.0        0.0  ...   0.0   0.0    0.0     0.0

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

# Ensure you have downloaded the required nltk data
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))

# Convert the set to a list
stop_words_list = list(stop_words)

# Stop Word Handling
vectorizer_sw = CountVectorizer(stop_words=stop_words_list)
X_sw = vectorizer_sw.fit_transform(emails)
df_sw = pd.DataFrame(X_sw.toarray(), columns=vectorizer_sw.get_feature_names_out())
print("Stop Word Handling:")
print(df_sw)



Stop Word Handling:
    10  10th  1st  able  achieved  achievement  action  additional  additions  \
0    0     0    0     1         0            0       0           0          0   
1    0     0    0     0         0            0       0           0          0   
2    1     0    0     0         0            0       0           0          0   
3    0     0    0     0         0            0       0           0          0   
4    0     0    0     0         0            0       0           0          0   
..  ..   ...  ...   ...       ...          ...     ...         ...        ...   
95   0     0    0     0         0            0       0           0          0   
96   0     0    0     0         0            0       0           0          0   
97   0     0    0     0         0            0       0           0          0   
98   0     0    0     0         0            0       0           0          0   
99   0     0    0     0         0            0       0           0          0   

    adj

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Stemming
ps = PorterStemmer()

def stem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [ps.stem(word) for word in tokens if word.isalpha()]
    return ' '.join(stemmed_tokens)

emails_stemmed = [stem_text(email) for email in emails]
vectorizer_stem = CountVectorizer()
X_stem = vectorizer_stem.fit_transform(emails_stemmed)
df_stem = pd.DataFrame(X_stem.toarray(), columns=vectorizer_stem.get_feature_names_out())
print("Stemming:")
print(df_stem)


Stemming:
    abl  about  achiev  action  addit  adjust  agenda  alic  am  an  ...  \
0     1      0       0       0      0       0       0     0   0   0  ...   
1     0      0       0       0      0       0       0     0   0   0  ...   
2     0      0       0       0      0       0       0     0   1   0  ...   
3     0      0       0       0      0       0       0     0   0   0  ...   
4     0      0       0       0      0       0       0     1   0   0  ...   
..  ...    ...     ...     ...    ...     ...     ...   ...  ..  ..  ...   
95    0      0       0       0      0       0       0     0   0   0  ...   
96    0      0       0       0      0       2       0     0   0   0  ...   
97    0      0       0       0      0       1       0     0   0   0  ...   
98    0      0       0       0      0       0       0     0   0   0  ...   
99    0      0       0       0      0       0       0     0   0   0  ...   

    window  with  work  xander  xena  yara  yesterday  you  your  zach  
0   

In [10]:
# Custom Tokenizer
def custom_tokenizer(text):
    return word_tokenize(text.lower())

vectorizer_token = CountVectorizer(tokenizer=custom_tokenizer)
X_token = vectorizer_token.fit_transform(emails)
df_token = pd.DataFrame(X_token.toarray(), columns=vectorizer_token.get_feature_names_out())
print("Custom Tokenizer:")
print(df_token)


Custom Tokenizer:
    !  'd  'll  'm  're  's  've  ,  .  10  ...  with  work  works  xander  \
0   0   0    0   0    1   0    0  2  3   0  ...     0     0      0       0   
1   0   0    0   0    0   1    0  2  4   0  ...     0     0      0       0   
2   0   0    0   0    0   0    0  2  3   1  ...     0     0      0       0   
3   0   0    1   0    0   0    0  2  3   0  ...     0     0      0       0   
4   0   0    0   0    0   0    0  2  3   0  ...     0     0      0       0   
.. ..  ..  ...  ..  ...  ..  ... .. ..  ..  ...   ...   ...    ...     ...   
95  0   0    1   0    0   0    0  2  2   0  ...     0     0      0       0   
96  0   0    0   0    0   0    0  2  3   0  ...     0     0      0       0   
97  0   0    1   0    0   0    0  2  2   0  ...     0     0      0       0   
98  0   0    0   0    1   0    0  2  3   0  ...     0     0      0       0   
99  0   0    0   0    0   0    0  2  3   0  ...     0     0      0       0   

    xena  yara  yesterday  you  your  zach  


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# Use normalized data for clustering
data = df_normalized

# KMeans Clustering
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_labels = kmeans.fit_predict(data)
print("KMeans Clustering Labels:")
print(kmeans_labels)

# EM Clustering
gmm = GaussianMixture(n_components=2, random_state=42)
gmm_labels = gmm.fit_predict(data)
print("EM Clustering Labels:")
print(gmm_labels)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans Clustering Labels:
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
EM Clustering Labels:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [12]:
# Add cluster labels to the dataframe
df_normalized['KMeans_Labels'] = kmeans_labels
df_normalized['EM_Labels'] = gmm_labels

# Summarize clusters
print("Cluster Summary - KMeans:")
print(df_normalized.groupby('KMeans_Labels').mean())

print("Cluster Summary - EM:")
print(df_normalized.groupby('EM_Labels').mean())


Cluster Summary - KMeans:
                     10      10th       1st  able     about  achieved  \
KMeans_Labels                                                           
0              0.014285  0.010101  0.010101   0.0  0.017495  0.010101   
1              0.000000  0.000000  0.000000   1.0  0.000000  0.000000   

               achievement    action  additional  additions  ...      work  \
KMeans_Labels                                                ...             
0                 0.014285  0.017495    0.010101   0.014285  ...  0.010101   
1                 0.000000  0.000000    0.000000   0.000000  ...  0.000000   

                  works    xander      xena      yara  yesterday       you  \
KMeans_Labels                                                                
0              0.010101  0.020202  0.014285  0.024742   0.010101  0.040474   
1              0.000000  0.000000  0.000000  0.000000   0.000000  0.235702   

                   your      zach  EM_Labels  
KMeans_L